In [9]:
import pickle
import numpy as np
from tqdm import tqdm

In [26]:
file_name = 'data_256.pkl'
image_data, sentiment_data, sentence_data, sentence_tags, hashtag_data = pickle.load(open(file_name, 'rb'))

In [27]:
size = 3000

image_data = image_data[:size]
sentiment_data = sentiment_data[:size]
sentence_data = sentence_data[:size]
sentence_tags = sentence_tags[:size]
hashtag_data = hashtag_data[:size]

In [28]:
print(image_data.shape)
print(sentiment_data.shape)
print(sentence_data.shape)
print(len(sentence_tags))

(3000, 256, 256, 3)
(3000, 4)
(3000,)
3000


In [29]:
input_tags = [sorted([tag[0] for tag in tags][:5]) for tags in sentence_tags]

In [30]:
from konlpy.tag import Komoran

komoran = Komoran()

output_tags = [[p[0] for p in komoran.pos(s)] for s in tqdm(sentence_data)]
[tags.insert(0, '\t') for tags in output_tags]
[tags.append('\n') for tags in output_tags]
None

100%|██████████| 3000/3000 [00:03<00:00, 933.24it/s] 


In [31]:
output_words = set()
for tags in output_tags:
    for tag in tags:
        if tag not in output_words:
            output_words.add(tag)
            
output_words = sorted(list(output_words))
output_words.insert(0, 'P')
n_output_words = len(output_words)

max_seq_length = max([len(_in) + len(_out) for _in, _out in zip(input_tags, output_tags)])

w2i_output = {w:i for i, w in enumerate(output_words)}
seq_list = []

for i, (_in, _out) in enumerate(zip(input_tags, output_tags)):
    for j in range(1, len(_out)):
        seq_list.append(_in + _out[:j+1])
        
n_seq_data = len(seq_list)

In [32]:
seq_data = np.zeros(shape=(n_seq_data, max_seq_length+1), dtype='int32')

for i, seq in enumerate(seq_list):
    seq_data[i, -len(seq):] = np.array([w2i_output[w] for w in seq])
    
x_seq_data = seq_data[:, :-1]

output_eye = np.eye(n_output_words)
y_seq_data = output_eye[seq_data[:, -1]]
# y_seq_data = seq_data[:, -1]

In [33]:
x_seq_data.shape, y_seq_data.shape

((129470, 152), (129470, 9948))

In [61]:
valid_size = 0
train_size = n_seq_data - valid_size
batch_size = 10000
n_batch = train_size // batch_size
n_valid = valid_size // batch_size

In [62]:
x_train, y_train = x_seq_data[:train_size], y_seq_data[:train_size]
x_valid, y_valid = x_seq_data[train_size:], y_seq_data[train_size:]

In [63]:
x_train.shape, y_train.shape

((129470, 152), (129470, 9948))

In [64]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN, LSTM

In [65]:
model = Sequential()
model.add(Embedding(n_output_words, 64, input_length=max_seq_length))
model.add(LSTM(128))
model.add(Dense(n_output_words, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [66]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 152, 64)           636672    
_________________________________________________________________
lstm_5 (LSTM)                (None, 128)               98816     
_________________________________________________________________
dense_5 (Dense)              (None, 9948)              1283292   
Total params: 2,018,780
Trainable params: 2,018,780
Non-trainable params: 0
_________________________________________________________________


In [68]:
model.fit(x_train, y_train, 
          validation_split=0.1,
          batch_size=batch_size,
          epochs=500)

Epoch 1/500
12/12 [==============================] - 5s 423ms/step - loss: 1.2136 - accuracy: 0.7480 - val_loss: 9.6644 - val_accuracy: 0.1648
Epoch 2/500
12/12 [==============================] - 4s 364ms/step - loss: 1.2153 - accuracy: 0.7473 - val_loss: 9.6746 - val_accuracy: 0.1643
Epoch 3/500
12/12 [==============================] - 4s 362ms/step - loss: 1.2131 - accuracy: 0.7478 - val_loss: 9.6873 - val_accuracy: 0.1647
Epoch 4/500
12/12 [==============================] - 4s 366ms/step - loss: 1.2127 - accuracy: 0.7486 - val_loss: 9.6911 - val_accuracy: 0.1641
Epoch 5/500
12/12 [==============================] - 4s 363ms/step - loss: 1.2286 - accuracy: 0.7436 - val_loss: 9.6564 - val_accuracy: 0.1638
Epoch 6/500
12/12 [==============================] - 4s 360ms/step - loss: 1.2314 - accuracy: 0.7430 - val_loss: 9.6857 - val_accuracy: 0.1637
Epoch 7/500
12/12 [==============================] - 4s 367ms/step - loss: 1.2219 - accuracy: 0.7452 - val_loss: 9.7076 - val_accuracy: 0.1627

Epoch 58/500
12/12 [==============================] - 4s 360ms/step - loss: 1.1635 - accuracy: 0.7575 - val_loss: 9.9127 - val_accuracy: 0.1623
Epoch 59/500
12/12 [==============================] - 4s 361ms/step - loss: 1.1584 - accuracy: 0.7589 - val_loss: 9.9241 - val_accuracy: 0.1620
Epoch 60/500
12/12 [==============================] - 4s 360ms/step - loss: 1.1517 - accuracy: 0.7607 - val_loss: 9.9281 - val_accuracy: 0.1629
Epoch 61/500
12/12 [==============================] - 4s 361ms/step - loss: 1.1481 - accuracy: 0.7615 - val_loss: 9.9274 - val_accuracy: 0.1617
Epoch 62/500
12/12 [==============================] - 4s 367ms/step - loss: 1.1462 - accuracy: 0.7623 - val_loss: 9.9369 - val_accuracy: 0.1616
Epoch 63/500
12/12 [==============================] - 4s 364ms/step - loss: 1.1442 - accuracy: 0.7623 - val_loss: 9.9512 - val_accuracy: 0.1607
Epoch 64/500
12/12 [==============================] - 4s 363ms/step - loss: 1.1420 - accuracy: 0.7632 - val_loss: 9.9438 - val_accuracy:

12/12 [==============================] - 4s 369ms/step - loss: 1.0879 - accuracy: 0.7749 - val_loss: 10.1379 - val_accuracy: 0.1612
Epoch 115/500
12/12 [==============================] - 4s 365ms/step - loss: 1.0870 - accuracy: 0.7746 - val_loss: 10.1624 - val_accuracy: 0.1610
Epoch 116/500
12/12 [==============================] - 4s 363ms/step - loss: 1.0851 - accuracy: 0.7750 - val_loss: 10.1746 - val_accuracy: 0.1611
Epoch 117/500
12/12 [==============================] - 4s 367ms/step - loss: 1.0839 - accuracy: 0.7758 - val_loss: 10.1822 - val_accuracy: 0.1614
Epoch 118/500
12/12 [==============================] - 4s 362ms/step - loss: 1.0840 - accuracy: 0.7759 - val_loss: 10.1907 - val_accuracy: 0.1605
Epoch 119/500
12/12 [==============================] - 4s 364ms/step - loss: 1.0823 - accuracy: 0.7757 - val_loss: 10.1916 - val_accuracy: 0.1606
Epoch 120/500
12/12 [==============================] - 4s 370ms/step - loss: 1.0801 - accuracy: 0.7763 - val_loss: 10.1758 - val_accuracy:

12/12 [==============================] - 4s 370ms/step - loss: 1.0355 - accuracy: 0.7848 - val_loss: 10.3999 - val_accuracy: 0.1583
Epoch 171/500
12/12 [==============================] - 4s 362ms/step - loss: 1.0325 - accuracy: 0.7861 - val_loss: 10.3993 - val_accuracy: 0.1595
Epoch 172/500
12/12 [==============================] - 4s 363ms/step - loss: 1.0311 - accuracy: 0.7864 - val_loss: 10.4082 - val_accuracy: 0.1592
Epoch 173/500
12/12 [==============================] - 4s 364ms/step - loss: 1.0302 - accuracy: 0.7865 - val_loss: 10.4184 - val_accuracy: 0.1584
Epoch 174/500
12/12 [==============================] - 4s 373ms/step - loss: 1.0296 - accuracy: 0.7864 - val_loss: 10.4116 - val_accuracy: 0.1594
Epoch 175/500
12/12 [==============================] - 4s 372ms/step - loss: 1.0285 - accuracy: 0.7864 - val_loss: 10.4122 - val_accuracy: 0.1594
Epoch 176/500
12/12 [==============================] - 4s 362ms/step - loss: 1.0278 - accuracy: 0.7871 - val_loss: 10.4235 - val_accuracy:

12/12 [==============================] - 4s 367ms/step - loss: 0.9756 - accuracy: 0.7988 - val_loss: 10.6467 - val_accuracy: 0.1593
Epoch 227/500
12/12 [==============================] - 4s 365ms/step - loss: 0.9751 - accuracy: 0.7987 - val_loss: 10.6570 - val_accuracy: 0.1584
Epoch 228/500
12/12 [==============================] - 4s 368ms/step - loss: 0.9749 - accuracy: 0.7990 - val_loss: 10.6496 - val_accuracy: 0.1593
Epoch 229/500
12/12 [==============================] - 4s 371ms/step - loss: 0.9752 - accuracy: 0.7983 - val_loss: 10.6788 - val_accuracy: 0.1587
Epoch 230/500
12/12 [==============================] - 4s 364ms/step - loss: 0.9740 - accuracy: 0.7990 - val_loss: 10.6699 - val_accuracy: 0.1587
Epoch 231/500
12/12 [==============================] - 4s 363ms/step - loss: 0.9748 - accuracy: 0.7980 - val_loss: 10.6755 - val_accuracy: 0.1587
Epoch 232/500
12/12 [==============================] - 4s 365ms/step - loss: 0.9765 - accuracy: 0.7976 - val_loss: 10.6951 - val_accuracy:

12/12 [==============================] - 4s 366ms/step - loss: 0.9290 - accuracy: 0.8087 - val_loss: 10.8946 - val_accuracy: 0.1566
Epoch 283/500
12/12 [==============================] - 4s 361ms/step - loss: 0.9267 - accuracy: 0.8085 - val_loss: 10.8971 - val_accuracy: 0.1575
Epoch 284/500
12/12 [==============================] - 4s 360ms/step - loss: 0.9249 - accuracy: 0.8096 - val_loss: 10.8979 - val_accuracy: 0.1578
Epoch 285/500
12/12 [==============================] - 4s 366ms/step - loss: 0.9264 - accuracy: 0.8091 - val_loss: 10.9064 - val_accuracy: 0.1574
Epoch 286/500
12/12 [==============================] - 4s 365ms/step - loss: 0.9255 - accuracy: 0.8095 - val_loss: 10.9063 - val_accuracy: 0.1578
Epoch 287/500
12/12 [==============================] - 4s 366ms/step - loss: 0.9243 - accuracy: 0.8101 - val_loss: 10.9177 - val_accuracy: 0.1572
Epoch 288/500
12/12 [==============================] - 4s 366ms/step - loss: 0.9226 - accuracy: 0.8098 - val_loss: 10.9145 - val_accuracy:

12/12 [==============================] - 4s 366ms/step - loss: 0.8838 - accuracy: 0.8186 - val_loss: 11.1159 - val_accuracy: 0.1575
Epoch 339/500
12/12 [==============================] - 4s 365ms/step - loss: 0.8833 - accuracy: 0.8183 - val_loss: 11.1246 - val_accuracy: 0.1572
Epoch 340/500
12/12 [==============================] - 4s 370ms/step - loss: 0.8810 - accuracy: 0.8194 - val_loss: 11.1246 - val_accuracy: 0.1571
Epoch 341/500
12/12 [==============================] - 4s 365ms/step - loss: 0.8794 - accuracy: 0.8194 - val_loss: 11.1272 - val_accuracy: 0.1572
Epoch 342/500
12/12 [==============================] - 4s 362ms/step - loss: 0.8787 - accuracy: 0.8200 - val_loss: 11.1301 - val_accuracy: 0.1574
Epoch 343/500
12/12 [==============================] - 4s 364ms/step - loss: 0.8788 - accuracy: 0.8197 - val_loss: 11.1373 - val_accuracy: 0.1577
Epoch 344/500
12/12 [==============================] - 4s 368ms/step - loss: 0.8777 - accuracy: 0.8196 - val_loss: 11.1307 - val_accuracy:

12/12 [==============================] - 4s 370ms/step - loss: 0.8389 - accuracy: 0.8284 - val_loss: 11.3439 - val_accuracy: 0.1570
Epoch 395/500
12/12 [==============================] - 4s 366ms/step - loss: 0.8372 - accuracy: 0.8289 - val_loss: 11.3505 - val_accuracy: 0.1564
Epoch 396/500
12/12 [==============================] - 4s 369ms/step - loss: 0.8370 - accuracy: 0.8290 - val_loss: 11.3367 - val_accuracy: 0.1581
Epoch 397/500
12/12 [==============================] - 4s 368ms/step - loss: 0.8470 - accuracy: 0.8258 - val_loss: 11.3366 - val_accuracy: 0.1569
Epoch 398/500
12/12 [==============================] - 4s 369ms/step - loss: 0.8433 - accuracy: 0.8273 - val_loss: 11.3565 - val_accuracy: 0.1575
Epoch 399/500
12/12 [==============================] - 4s 362ms/step - loss: 0.8475 - accuracy: 0.8259 - val_loss: 11.3635 - val_accuracy: 0.1569
Epoch 400/500
12/12 [==============================] - 4s 368ms/step - loss: 0.8464 - accuracy: 0.8259 - val_loss: 11.3455 - val_accuracy:

12/12 [==============================] - 4s 369ms/step - loss: 0.8069 - accuracy: 0.8349 - val_loss: 11.5448 - val_accuracy: 0.1561
Epoch 451/500
12/12 [==============================] - 4s 365ms/step - loss: 0.8029 - accuracy: 0.8364 - val_loss: 11.5820 - val_accuracy: 0.1553
Epoch 452/500
12/12 [==============================] - 4s 362ms/step - loss: 0.7987 - accuracy: 0.8377 - val_loss: 11.5738 - val_accuracy: 0.1560
Epoch 453/500
12/12 [==============================] - 4s 368ms/step - loss: 0.7953 - accuracy: 0.8386 - val_loss: 11.5759 - val_accuracy: 0.1558
Epoch 454/500
12/12 [==============================] - 4s 368ms/step - loss: 0.7956 - accuracy: 0.8385 - val_loss: 11.5718 - val_accuracy: 0.1559
Epoch 455/500
12/12 [==============================] - 4s 366ms/step - loss: 0.7970 - accuracy: 0.8376 - val_loss: 11.6026 - val_accuracy: 0.1552
Epoch 456/500
12/12 [==============================] - 4s 372ms/step - loss: 0.7974 - accuracy: 0.8380 - val_loss: 11.5857 - val_accuracy:

In [70]:
file_model = 'model_ver7.hdf5'
model.save(file_model)

In [13]:
from tensorflow.keras.models import load_model
file_model = 'model_ver7.hdf5'
model = load_model(file_model)

In [71]:
def sequence_generation(model, tags):
    tags = tags.copy()
    tags.append('\t')
    sentence = ''
    while tags[-1] != '\n' and len(tags) < max_seq_length:
        x_test = np.zeros(shape=(1, max_seq_length), dtype='int32')
        x_test[0, -len(tags):] = [w2i_output[w] for w in tags]
        result = model.predict_classes(x_test, verbose=0)
        tags.append(output_words[result[0]])
        sentence += ' ' + tags[-1]
    return sentence.replace('\n', '')

In [73]:
start_idx = 0
end_idx = start_idx + 20
for ori, tags in zip(sentence_data[start_idx:end_idx], input_tags[start_idx:end_idx]):
    print('원본')
    print(ori[1:-1])
    print(tags)
    print('예측')
    print(sequence_generation(model, tags))
    print()

원본
맛집 오늘의 포인트 이없다는거
['맛집', '없', '오늘', '이', '포인트']
예측
 누나 거기 다가 컴백 홈 트 자켓으로 오늘 도 하루 도 별 이 들 어서 우리 는 독 박 육아 한 상 이 다른 최 고급 미러 급 제품 직접 받 아 보 시 어요 우드 행 잉 액자 품절 시 행사 가 종료 되 ㅂ니다 예약 은 브레드 세븐 에서 준비 하 아 보 시 어요 

원본
오늘 하늘 너무 이쁘다
['너무', '오늘', '이쁘', '하늘']
예측
 하늘 보 면서 무슨 생각 보다 

원본
우오오오오옹 전역 일차 날씨 짱좋다아아아아
['오오', '오오', '옹', '우', '전역']
예측
 이제 아침 이 돌 아서 아침 도 너무 좋 아 지 고 피곤 하 ㄹ 때 마다 항상 쓰 어 먹 었 어요 쨍 쨍 햇빛 에 선글라스 필수 죠 보통 패션 선글라스 는 운동 하 기 간편 하 게 

원본
하늘이너무이쁘다
['너무', '이쁘', '하늘']
예측
 하늘 이 너무 이쁘 다 

원본
하늘이 맑고 푸르다
['맑', '푸르', '하늘']
예측
 하늘 이 너무 이쁘 ㄴ 곳 

원본
얼마만에 이런 하늘이야 넘 좋다
['넘', '얼마', '이런', '좋', '하늘']
예측
 얼마 만 에 보 고 싶 다 

원본
오랜만에 맑은 하늘
['맑', '오랜만', '하늘']
예측
 오늘 날씨 미치 었 다 산 에 가 아서 팩 이나 하 아 주 어서 좋 으시 ㄴ 분 들 에게 주 시 었 습니다 웃음 이 끊기 지 않 았 습니다 방문 전 함께 분 의 교감 중 

원본
오우 오늘 무슨일 이쁘다증말 잠시김밥사러 산책 쫌만걸어도배가너무땡겨
['무슨', '오늘', '오우', '이쁘', '일']
예측
 오우 오늘 무슨 일 만 산 롭 어 지 어 지나 ㄴ 시간 이 열심히 채우 어 먹 을 수 있 어서 좋 아요 

원본
오늘 하늘이 너무 이뻐서 시간차로 계속 찍게되네
['너무', '시간', '오늘', '이쁘', '하늘']
예측
 하늘 보 면서 무슨 생각 하 고 있 는데 금식 하 고 있 는데 금식 하 고 있 는데 금식 하 고 싶 다 

원본


['맛집', '오늘', '포인트', '이', '없', '거']

In [58]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

In [125]:
tf.reset_default_graph()

n_embedding = 64
n_hidden = 64
learning_rate = 1e-2

X = tf.placeholder(tf.int32, shape=(None, max_seq_length))
Y = tf.placeholder(tf.int32, shape=(None,))
# Y = tf.placeholder(tf.float32, shape=(None,))

embedding = tf.Variable(tf.random_uniform((n_output_words, n_embedding), -1, 1))
embed = tf.nn.embedding_lookup(embedding, X)

cell = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)
outputs, states = tf.nn.dynamic_rnn(cell, embed, dtype=tf.float32)

model = tf.layers.dense(tf.reshape(outputs, [-1, max_seq_length*n_hidden]), n_output_words, activation=tf.nn.softmax)
# cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels=Y))
cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

predict = tf.arg_max(model, 1)

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.


In [126]:
with tf.device('/gpu:0'):
    sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True, 
                                            log_device_placement=True))
    sess.run(tf.global_variables_initializer())

Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce RTX 2080 Ti, pci bus id: 0000:68:00.0, compute capability: 7.5



In [127]:
tf.trainable_variables()

[<tf.Variable 'Variable:0' shape=(6883, 64) dtype=float32_ref>,
 <tf.Variable 'rnn/basic_lstm_cell/kernel:0' shape=(128, 256) dtype=float32_ref>,
 <tf.Variable 'rnn/basic_lstm_cell/bias:0' shape=(256,) dtype=float32_ref>,
 <tf.Variable 'dense/kernel:0' shape=(14848, 6883) dtype=float32_ref>,
 <tf.Variable 'dense/bias:0' shape=(6883,) dtype=float32_ref>]

In [128]:
total_epoch = 100

# for epoch in tqdm(range(total_epoch)):
for epoch in range(total_epoch):
    total_loss = 0
    for batch in tqdm(range(n_batch)):
#     for batch in range(n_batch):
        batch_xs = x_train[batch*batch_size:(batch+1)*batch_size]
        batch_ys = y_train[batch*batch_size:(batch+1)*batch_size]
        
        _, loss = sess.run([optimizer, cost], feed_dict={
                X: batch_xs,
                Y: batch_ys
            })
        
        total_loss += loss
    
    total_valid_loss = 0
    for batch in range(n_valid):
        valid_xs = x_valid[batch*batch_size:(batch+1)*batch_size]
        valid_ys = y_valid[batch*batch_size:(batch+1)*batch_size]
        
        val_loss = sess.run(cost, feed_dict={
                X: valid_xs,
                Y: valid_ys
            })
        total_valid_loss += val_loss
    
    print('Epoch: %04d Cost: %.6f Valid Cost: %.6f' %(epoch + 1, total_loss / n_batch, total_valid_loss / n_valid))

  0%|          | 0/22 [00:00<?, ?it/s]

Epoch: 0001 Cost: 8.803528 Valid Cost: 8.801259


  0%|          | 0/22 [00:00<?, ?it/s]

Epoch: 0002 Cost: 8.800805 Valid Cost: 8.801259


  0%|          | 0/22 [00:00<?, ?it/s]

Epoch: 0003 Cost: 8.800805 Valid Cost: 8.801259


 45%|████▌     | 10/22 [00:02<00:03,  3.88it/s]


KeyboardInterrupt: 

In [115]:
def translate(tags):
    x_test = np.zeros(shape=(1, max_seq_length), dtype='float32')
    tags = tags + ['\t']
    idx = -len(tags)
    x_test[0, idx:] = [w2i_output[w] for w in tags]
    while x_test[0, -1] != w2i_output['\n'] and x_test[0][0] == 0:
        print(idx)
        pred = sess.run(predict, feed_dict={X: x_test})[0]
        x_test[0, idx-1:-1] = x_test[0, idx:]
        x_test[0, -1] = pred
        idx -= 1
    end = np.where(x_test[0]==w2i_output['\n'])[0]
    end = end[0] if end.shape[0] > 0 else -1
    ret = x_test[0, np.where(x_test[0]==w2i_output['\t'])[0][0]:end]
    return ''.join([output_words[ret]])

In [121]:
tags = input_tags[0]
x_test = np.zeros(shape=(1, max_seq_length), dtype='float32')
tags = tags + ['\t']
idx = -len(tags)
x_test[0, idx:] = [w2i_output[w] for w in tags]
while x_test[0, -1] != w2i_output['\n'] and x_test[0][0] == 0:
    print(idx)
    pred = sess.run(predict, feed_dict={X: x_test})[0]
    x_test[0, idx-1:-1] = x_test[0, idx:]
    x_test[0, -1] = pred
    idx -= 1
end = np.where(x_test[0]==w2i_output['\n'])[0]
end = end[0] if end.shape[0] > 0 else -1
ret = x_test[0, np.where(x_test[0]==w2i_output['\t'])[0][0]:end]
''.join([output_words[map(int, ret)]])

-7
-8
-9
-10
-11
-12
-13
-14
-15
-16
-17
-18
-19
-20
-21
-22
-23
-24
-25
-26
-27
-28
-29
-30
-31
-32
-33
-34
-35
-36
-37
-38
-39
-40
-41
-42
-43
-44
-45
-46
-47
-48
-49
-50
-51
-52
-53
-54
-55
-56
-57
-58
-59
-60
-61
-62
-63
-64
-65
-66
-67
-68
-69
-70
-71
-72
-73
-74
-75
-76
-77
-78
-79
-80
-81
-82
-83
-84
-85
-86
-87
-88
-89
-90
-91
-92
-93
-94
-95
-96
-97
-98
-99
-100
-101
-102
-103
-104
-105
-106
-107
-108
-109
-110
-111
-112
-113
-114
-115
-116
-117
-118


KeyboardInterrupt: 

In [124]:
''.join([output_words[int(i)] for i in ret])

'\t하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하'